In this exercise, we will first create an extension of the IoU- and ReID-based tracker we created in the previous exercise that will make it more robust to occlusions by allowing it to recover from missed detections.

We will then implement a Message Passing Network that we will use to learn features that combine position and appearance information. On these edge features, we then base our prediction on associations between past tracks and new detections.

Your tasks are the following:
- Adapt the track management scheme of our Tracker to allow it to recover from missed detections.
- Implement a Message Passing Network to operate on bipartite graphs
- Implement the pairwise feature computation to obtain features for our Message Passing Network
- Train the Message Passing Network and improve your tracker's IDF1 score
- Use the Message Passing Network instead of the distance matrix in the Tracker

##  Setup
### Download and extract project data to your Google Drive

1. **Required**: Please follow all instructions of exercise 0 and exercise 1 before running this notebook.
1. Save this notebook to your Google Drive by clicking Save a copy in Drive from the File menu.
3. Download [this](https://vision.in.tum.de/webshare/u/cetintas/cv3dst_gnn_exercise.zip) zip file to your desktop, extract it and upload it into the Colab Notebooks folder in your Google Drive.

#### Connect the notebook to your Google Drive

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
root_dir = "gdrive/My Drive/Colab Notebooks/cv3dst_exercise/"
gnn_root_dir = "gdrive/My Drive/Colab Notebooks/cv3dst_gnn_exercise/"

The `root_dir` path points to the directory and the content in your Google Drive.

In [3]:
!ls "gdrive/My Drive/Colab Notebooks/cv3dst_gnn_exercise/data"

preprocessed_data_test_2.pth  preprocessed_data_train_2.pth


#### Install and import Python libraries

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

!pip install tqdm lap
!pip install https://github.com/timmeinhardt/py-motmetrics/archive/fix_pandas_deprecating_warnings.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 34.6 MB/s 
  Created wheel for lap: filename=lap-0.4.0-cp38-cp38-linux_x86_64.whl size=1619803 sha256=e4f3918aaea74de412acfa36a0a6c1521e8ccb4491b37a3f29b80050159d47c6
  Stored in directory: /root/.cache/pip/wheels/86/ec/8a/2baa6fb3ea3a3bdc40a9c2c4b8957b5b544f207531f7a78a89
Successfully built lap
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     | 148 kB 4.0 MB/s
  Created wheel for motmetrics: filename=motmetrics-1.1.3-py3-none-any.whl size=134196 sha256=338eca105a2e6c26b9966c477450ffd295a1fe84eebdf55fff852234980e54a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-ahtd56dw/wheels/ed/d8/67/5ec51b9b586679a5db9d1c943ec461fc42d9e2c0aeb82c86dd
Successfully built motmetrics


In [5]:
import os
import sys
sys.path.append(os.path.join(gnn_root_dir, 'src'))


import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm.autonotebook import tqdm

import torch
from torch.utils.data import DataLoader

from tracker.data_track import MOT16Sequences
from tracker.tracker import Tracker, ReIDTracker
from tracker.utils import run_tracker, cosine_distance
from scipy.optimize import linear_sum_assignment as linear_assignment
import os.path as osp

import motmetrics as mm
mm.lap.default_solver = 'lap'

<ipython-input-5-2d293b2d3d98>:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
!ls "gdrive/My Drive/Colab Notebooks/cv3dst_exercise/data/MOT16/train"
!ls "gdrive/My Drive/Colab Notebooks/cv3dst_exercise/data/MOT16/test"

MOT16-02  MOT16-04  MOT16-05  MOT16-09	MOT16-10  MOT16-11  MOT16-13
MOT16-01  MOT16-03  MOT16-06  MOT16-07	MOT16-08  MOT16-12  MOT16-14


## Speed-Ups
In order to speed up training and inference runtimes, in this exercise we will be working with pre-computed detections and ReID embeddings. We ran the object detector we provided in Exercise 0 and applied to all frames. We also computed reid embeddings for all boxes in every frame of the dataset so that they don't need to be computed every time you run your tracker. This yields over 10x speed improvements. You will not have to work directly with the resulting files, as we have internally adapted the boilerplate code to work with them.

In [6]:
train_db = torch.load(osp.join(gnn_root_dir, 'data/preprocessed_data_train_2.pth'))

## Exercise Part 0 - Assignment's 1 ReIDHungarianTracker

We start by providing a sample solution of the ``ReIDTracker`` from Exercise 1.
It will serve as our baseline.


Recall that this tracker works by performing frame-to-frame bipartite matching between newly detected boxes and past tracks based on ReID distance. Whenever a past track cannot be matched, its killed. And whenever, a newly detected box cannot be match, it starts a new trajectory.

**NOTE**: We have modified the ``compute_distance`` function in ``data_association`` from last week to include a threshold on ReID distance (if ReID distance >0.1, matching is not possible). This is important to prevent our tracker from reusing tracks for very dissimilar objects.

Results:
```
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP
MOT16-02 40.5% 57.5% 31.3% 52.2% 96.1%  62  12  38 12  390  8873 203  210 49.1% 0.090
MOT16-05 54.4% 64.4% 47.1% 68.8% 94.0% 133  54  67 12  305  2156 330  149 59.7% 0.142
MOT16-09 49.9% 61.7% 41.9% 66.3% 97.7%  26  13  12  1   82  1793  77   76 63.3% 0.082
MOT16-11 61.1% 67.4% 55.9% 80.2% 96.6%  75  44  24  7  266  1871 176   91 75.5% 0.083
OVERALL  49.6% 62.3% 41.2% 63.5% 96.1% 296 123 141 32 1043 14693 786  526 59.0% 0.097
```


In [7]:
_UNMATCHED_COST=255
class ReIDHungarianTracker(ReIDTracker):
    def data_association(self, boxes, scores, pred_features):  
        """Refactored from previous implementation to split it onto distance computation and track management"""
        if self.tracks:
            track_boxes = torch.stack([t.box for t in self.tracks], axis=0)
            track_features = torch.stack([t.get_feature() for t in self.tracks], axis=0)
            
            distance = self.compute_distance_matrix(track_features, pred_features,
                                                    track_boxes, boxes, metric_fn=cosine_distance)

            # Perform Hungarian matching.
            row_idx, col_idx = linear_assignment(distance)            
            self.update_tracks(row_idx, col_idx, distance, boxes, scores, pred_features)

            
        else:
            # No tracks exist.
            self.add(boxes, scores, pred_features)
        
    def update_tracks(self, row_idx, col_idx, distance, boxes, scores, pred_features):
        """Updates existing tracks and removes unmatched tracks.
           Reminder: If the costs are equal to _UNMATCHED_COST, it's not a 
           match. 
        """
        track_ids = [t.id for t in self.tracks]

        unmatched_track_ids = []
        seen_track_ids = []
        seen_box_idx = []
        for track_idx, box_idx in zip(row_idx, col_idx):
            costs = distance[track_idx, box_idx] 
            internal_track_id = track_ids[track_idx]
            seen_track_ids.append(internal_track_id)
            if costs == _UNMATCHED_COST:
                unmatched_track_ids.append(internal_track_id)
            else:
                self.tracks[track_idx].box = boxes[box_idx]
                self.tracks[track_idx].add_feature(pred_features[box_idx])
                seen_box_idx.append(box_idx)

        unseen_track_ids = set(track_ids) - set(seen_track_ids)
        unmatched_track_ids.extend(list(unseen_track_ids))
        self.tracks = [t for t in self.tracks
                       if t.id not in unmatched_track_ids]


        # Add new tracks.
        new_boxes_idx = set(range(len(boxes))) - set(seen_box_idx)
        new_boxes = [boxes[i] for i in new_boxes_idx]
        new_scores = [scores[i] for i in new_boxes_idx]
        new_features = [pred_features[i] for i in new_boxes_idx]
        self.add(new_boxes, new_scores, new_features)


In [8]:
val_sequences = MOT16Sequences('MOT16-reid', root_dir = osp.join(root_dir, 'data/MOT16'), vis_threshold=0.)

In [9]:
tracker = ReIDHungarianTracker(None)
run_tracker(val_sequences, db=train_db, tracker=tracker, output_dir=None)

Tracking: MOT16-02
Tracks found: 367
Runtime for MOT16-02: 5.3 s.
Tracking: MOT16-05
Tracks found: 464
Runtime for MOT16-05: 2.1 s.
Tracking: MOT16-09
Tracks found: 98
Runtime for MOT16-09: 1.2 s.
Tracking: MOT16-11
Tracks found: 292
Runtime for MOT16-11: 2.7 s.
Runtime for all sequences: 11.4 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP
MOT16-02 40.5% 57.5% 31.3% 52.2% 96.1%  62  12  38 12  390  8873 203  210 49.1% 0.090
MOT16-05 54.4% 64.4% 47.1% 68.8% 94.0% 133  54  67 12  305  2156 330  149 59.7% 0.142
MOT16-09 49.9% 61.7% 41.9% 66.3% 97.7%  26  13  12  1   82  1793  77   76 63.3% 0.082
MOT16-11 61.1% 67.4% 55.9% 80.2% 96.6%  75  44  24  7  266  1871 176   91 75.5% 0.083
OVERALL  49.6% 62.3% 41.2% 63.5% 96.1% 296 123 141 32 1043 14693 786  526 59.0% 0.097


,idf1,idp,idr,recall,precision,num_unique_objects,mostly_tracked,partially_tracked,mostly_lost,num_false_positives,num_misses,num_switches,num_fragmentations,mota,motp
MOT16-02,0.405105,0.575262,0.312631,0.522469,0.961378,62,12,38,12,390,8873,203,210,0.490555,0.090340
MOT16-05,0.544104,0.643506,0.471303,0.688304,0.939795,133,54,67,12,305,2156,330,149,0.596501,0.142002
MOT16-09,0.499161,0.617322,0.418967,0.663286,0.977310,26,13,12,1,82,1793,77,76,0.633427,0.082336
MOT16-11,0.611340,0.673988,0.559347,0.801717,0.966032,75,44,24,7,266,1871,176,91,0.754875,0.082736
OVERALL,0.495843,0.623022,0.411784,0.635038,0.960803,296,123,141,32,1043,14693,786,526,0.589607,0.096605


## Exercise Part I - Long-Term ReID Tracker


The tracker above has an obvious limitation: whenever a track cannot be matched with the detections of a given frame the track will be killed. This means that if our detector misses an object in a single frame (due to e.g. occlusion), we will not be able to recover that track, and we will start a new one. 

To fix this issue, we would like to allow our tracker to maintain tracks that are not matched during data association. We will refer to these tracks as **inactive**. During data association, we will try to match the detected boxes for the current frame to both tracks that are active (i.e. tracks that we were able to match in the previous frame) as well as those that are inactive. Therefore, if a detector misses an object in a frame and the object reappears after a few frames, we will still be able to match it to its corresponding track, instead of creating a new one.

In order to adapt our tracker to have this behavior, we will use the `inactive` attribute from the `track` class (see `tracker/tracker.py`. This attribute will be assigned an integer indicating for how many frames a track has remained unmatched. Whenever we are able to match the track `t`, we will set `t.inactive=0` and, naturally, when tracks are initialized, the class constructor sets `inactive=0`. 

Your job is to maintain the `inactive` attribute of all tracks being kept by tracker so that its value represents the number of frames for which the track has been unmatched. Additionally, we introduce a `patience` parameter. Whenever a track has been inactive for more than `inactive` frames. it will need to be killed.

Results should approximately be around:

```
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP
MOT16-02 45.9% 65.1% 35.4% 52.2% 96.1%  62  12  37 13  390  8873 130  210 49.4% 0.090
MOT16-05 63.4% 75.0% 54.9% 68.8% 94.0% 133  54  67 12  305  2156 283  149 60.3% 0.142
MOT16-09 52.5% 64.9% 44.1% 66.3% 97.7%  26  13  12  1   82  1793  49   76 63.9% 0.083
MOT16-11 68.3% 75.3% 62.5% 80.2% 96.6%  75  44  24  7  266  1871 136   90 75.9% 0.083
OVERALL  55.7% 70.0% 46.2% 63.5% 96.1% 296 123 140 33 1043 14693 598  525 59.4% 0.097
```


In [10]:
class LongTermReIDHungarianTracker(ReIDHungarianTracker):
    def __init__(self, patience, *args, **kwargs):
        """ Add a patience parameter"""
        self.patience=patience
        super().__init__(*args, **kwargs)

    def update_results(self):
        """Only store boxes for tracks that are active"""
        for t in self.tracks:
            if t.id not in self.results.keys():
                self.results[t.id] = {}
            if t.inactive == 0: # Only change
                self.results[t.id][self.im_index] = np.concatenate([t.box.cpu().numpy(), np.array([t.score])])

        self.im_index += 1        
        
    def update_tracks(self, row_idx, col_idx, distance, boxes, scores, pred_features):
        track_ids = [t.id for t in self.tracks]

        unmatched_track_ids = []
        seen_track_ids = []
        seen_box_idx = []
        for track_idx, box_idx in zip(row_idx, col_idx):
            costs = distance[track_idx, box_idx] 
            internal_track_id = track_ids[track_idx]
            seen_track_ids.append(internal_track_id)
            if costs == _UNMATCHED_COST:
                unmatched_track_ids.append(internal_track_id)

            else:
                self.tracks[track_idx].box = boxes[box_idx]
                self.tracks[track_idx].add_feature(pred_features[box_idx])
                
                # Note: the track is matched, therefore, inactive is set to 0
                self.tracks[track_idx].inactive=0
                seen_box_idx.append(box_idx)
                

        unseen_track_ids = set(track_ids) - set(seen_track_ids)
        unmatched_track_ids.extend(list(unseen_track_ids))
        ##################
        ### TODO starts
        ##################
        
        # Update the `inactive` attribute for those tracks that have been 
        # not been matched. kill those for which the inactive parameter 
        # is > self.patience
        killed_tracks = []
        for t in self.tracks:
            if t.id in unmatched_track_ids:
                t.inactive += 1
            if t.inactive > self.patience:
                killed_tracks.append(t.id)
        self.tracks = [t for t in self.tracks
                       if t.id not in killed_tracks] # <-- Needs to be updated
        
        ##################
        ### TODO ends
        ##################        
        
        new_boxes_idx = set(range(len(boxes))) - set(seen_box_idx)
        new_boxes = [boxes[i] for i in new_boxes_idx]
        new_scores = [scores[i] for i in new_boxes_idx]
        new_features = [pred_features[i] for i in new_boxes_idx]
        self.add(new_boxes, new_scores, new_features)

In [ ]:
tracker = LongTermReIDHungarianTracker(patience=20, obj_detect=None)
run_tracker(val_sequences, db=train_db, tracker=tracker, output_dir=None)

Tracking: MOT16-02
Tracks found: 207
Runtime for MOT16-02: 6.2 s.
Tracking: MOT16-05
Tracks found: 355
Runtime for MOT16-05: 2.2 s.
Tracking: MOT16-09
Tracks found: 65
Runtime for MOT16-09: 1.5 s.
Tracking: MOT16-11
Tracks found: 217
Runtime for MOT16-11: 3.3 s.
Runtime for all sequences: 13.1 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP
MOT16-02 45.9% 65.1% 35.4% 52.2% 96.1%  62  12  37 13  390  8873 130  210 49.4% 0.090
MOT16-05 63.4% 75.0% 54.9% 68.8% 94.0% 133  54  67 12  305  2156 283  149 60.3% 0.142
MOT16-09 52.5% 64.9% 44.1% 66.3% 97.7%  26  13  12  1   82  1793  49   76 63.9% 0.083
MOT16-11 68.3% 75.3% 62.5% 80.2% 96.6%  75  44  24  7  266  1871 136   90 75.9% 0.083
OVERALL  55.7% 70.0% 46.2% 63.5% 96.1% 296 123 140 33 1043 14693 598  525 59.4% 0.097


,idf1,idp,idr,recall,precision,num_unique_objects,mostly_tracked,partially_tracked,mostly_lost,num_false_positives,num_misses,num_switches,num_fragmentations,mota,motp
MOT16-02,0.458524,0.651119,0.353856,0.522469,0.961378,62,12,37,13,390,8873,130,210,0.494484,0.090339
MOT16-05,0.633898,0.749704,0.549082,0.688304,0.939795,133,54,67,12,305,2156,283,149,0.603296,0.141942
MOT16-09,0.525115,0.649419,0.440751,0.663286,0.977310,26,13,12,1,82,1793,49,76,0.638685,0.082643
MOT16-11,0.683269,0.753288,0.625159,0.801717,0.966032,75,44,24,7,266,1871,136,90,0.759114,0.082754
OVERALL,0.556888,0.699726,0.462480,0.635038,0.960803,296,123,140,33,1043,14693,598,525,0.594277,0.096641


## Exercise Part II - Building a tracker based on Neural Message Passing

Our ``LongTermReIDHungarianTracker`` is still limited when compared to current modern trackers. 

Firstly, it relies solely on appearance to predict similarity scores between objectes. This can be problematic whenever appearance alone may not discriminative, and it'd be best to also take into account object position and size attributes. Secondly, our tracker can only account for pairwise similarities among objects. Ideally, we would like it to also consider higher-order information.

To address these limitations. We will now build a tracker that will combine both apperance and position information with a Message Passing Neural Network, inspired by the approach presented in [Learning a Neural Solver for Multiple Object Tracking, CVPR 2020](https://arxiv.org/abs/1912.07515)

The overall idea will be to build, for every tracking step, a bipartite graph containing two sets of nodes: past tracks, and detections in the current frame. We will initialize node features with ReID embeddings, and edge features with relative position features and ReID distance. We will use an MPN to refine these edge embeddings. The learning task will be to classify the edge embeddings in this graph, which is equivalent to predicting the entries of our data association similarity matrix.


### Building an MPN for Bipartite Graphs

We will first build a Neural Message Passing layer based on the Graph Networks framework introduced in [Relational inductive biases, deep learning, and graph networks, arXiv 2020](https://arxiv.org/abs/1806.01261), as explained in the *A More General Framework* slides of [Lecture 5](https://youtu.be/BR3Y5bAz5Dw) (slides 70 to 75).

We will be using a bipartite graph, i.e., we will have two sets of nodes $A$ (past tracks), and $B$ (detections), and our set of edges will be $A\times B$. That is, we will connect every pair of past tracks and detections.

We will have initial node features (i.e. reid embeddings) matrices: $X_A$ and $X_B$ and an initial edge features tensor $E$.

$X_A$ and $X_B$ have shape $|A|\times \text{node_dim}$ and $|B|\times \text{node_dim}$, respectively.

$E$ has shape $|A| \times |B| \times \text{edge_dim}$. Its $(i, j)$ entry contains the edge features of node $i$ in $A$ and node $j$ in $B$.

With the given layer, we will produce new node feature matrices $X_A'$ and $X_B'$ and edge features $E'$ with the same dimensions. 
Please refer to the formulas in the slides and figure how to apply them in this setting.

You are asked to implement both the node and edge update steps in the class below

**NOTE 1**: Working with a bipartite graph allows us to vectorize all operations in the formulas in a straightforward manner (keep in mind that we store edge features in a matrix). Given a node in $A$, it is connected to all nodes in $B$.

**NOTE 2**: You do not need to care about batching several graphs. This implementation will only work with a single graph at a time.

In [11]:
from torch import nn

class BipartiteNeuralMessagePassingLayer(nn.Module):    
    def __init__(self, node_dim, edge_dim, dropout=0.):
        super().__init__()

        edge_in_dim  = 2*node_dim + 2*edge_dim # 2*edge_dim since we always concatenate initial edge features
        self.edge_mlp = nn.Sequential(*[nn.Linear(edge_in_dim, edge_dim), nn.ReLU(), nn.Dropout(dropout), 
                                    nn.Linear(edge_dim, edge_dim), nn.ReLU(), nn.Dropout(dropout)])

        node_in_dim  = node_dim + edge_dim
        self.node_mlp = nn.Sequential(*[nn.Linear(node_in_dim, node_dim), nn.ReLU(), nn.Dropout(dropout),  
                                        nn.Linear(node_dim, node_dim), nn.ReLU(), nn.Dropout(dropout)])

    def edge_update(self, edge_embeds, nodes_a_embeds, nodes_b_embeds):
        """
        Node-to-edge updates, as descibed in slide 71, lecture 5.
        Args:
            edge_embeds: torch.Tensor with shape (|A|, |B|, 2 x edge_dim) 
            nodes_a_embeds: torch.Tensor with shape (|A|, node_dim)
            nodes_a_embeds: torch.Tensor with shape (|B|, node_dim)
            
        returns:
            updated_edge_feats = torch.Tensor with shape (|A|, |B|, edge_dim) 
        """
        
        n_nodes_a, n_nodes_b, _  = edge_embeds.shape
        
        ########################
        #### TODO starts
        ########################
        nodes_a_embeds = nodes_a_embeds[:, None, :].repeat(1, n_nodes_b, 1)
        nodes_b_embeds = nodes_b_embeds[None, :, :].repeat(n_nodes_a, 1, 1)
        edge_in = torch.cat((nodes_a_embeds, edge_embeds, nodes_b_embeds), dim=-1).cuda() # has shape (|A|, |B|, 2*node_dim + 2*edge_dim) 
        
        ########################
        #### TODO ends
        ########################
        
        
        return self.edge_mlp(edge_in)

    def node_update(self, edge_embeds, nodes_a_embeds, nodes_b_embeds):
        """
        Edge-to-node updates, as descibed in slide 75, lecture 5.

        Args:
            edge_embeds: torch.Tensor with shape (|A|, |B|, edge_dim) 
            nodes_a_embeds: torch.Tensor with shape (|A|, node_dim)
            nodes_b_embeds: torch.Tensor with shape (|B|, node_dim)
            
        returns:
            tuple(
                updated_nodes_a_embeds: torch.Tensor with shape (|A|, node_dim),
                updated_nodes_b_embeds: torch.Tensor with shape (|B|, node_dim)
                )
        """
        
        ########################
        #### TODO starts
        ########################
        
        # NOTE: Use 'sum' as aggregation function
        m_a = torch.sum(edge_embeds, dim=1) # has shape A x edge_dim
        m_b = torch.sum(edge_embeds, dim=0) # has shape B x edge_dim
        nodes_a_in = torch.cat((m_a, nodes_a_embeds), dim=-1).cuda() # Has shape (|A|, node_dim + edge_dim) 
        nodes_b_in = torch.cat((m_b, nodes_b_embeds), dim=-1).cuda() # Has shape (|B|, node_dim + edge_dim) 
        ########################
        #### TODO ends
        ########################

        nodes_a = self.node_mlp(nodes_a_in)
        nodes_b = self.node_mlp(nodes_b_in)

        return nodes_a, nodes_b

    def forward(self, edge_embeds, nodes_a_embeds, nodes_b_embeds):
        edge_embeds_latent = self.edge_update(edge_embeds, nodes_a_embeds, nodes_b_embeds)
        nodes_a_latent, nodes_b_latent = self.node_update(edge_embeds_latent, nodes_a_embeds, nodes_b_embeds)

        return edge_embeds_latent, nodes_a_latent, nodes_b_latent

## Building the entire network to predict similarities
We now build the network that generates initial node and edge features, performs neural message passing, and classifies edges in order to produce the final costs that we will use for data association.

You need to implement the method that computes the initial edge features. You can follow [1] and, given a two bounding boxes $(x_i, y_i, w_i, h_i)$ and  $(x_j, y_j, w_j, h_j)$ and timestamps $t_i$ and $t_j$, compute an initial 5-dimensional edge feature vector as:
$$ E_{(i, j)} = \left(\frac{2(x_j - x_i)}{h_i + h_j}, \frac{2(y_j - y_i)}{h_i + h_j}, \log{\frac{h_i}{h_j}}, \log{\frac{w_i}{w_j}}, t_j - t_i \right )$$


Feel free to engineer your own features (e.g. use IoU, etc.)

In [12]:
import copy
def change_coords(coord):
    new_coord = copy.deepcopy(coord)
    new_coord[:, 2] = coord[:, 2] - coord[:, 0] # width
    new_coord[:, 3] = coord[:, 3] - coord[:, 1] # heigth
    new_coord[:, 0] = coord[:, 0] + new_coord[:, 2] / 2 # x center
    new_coord[:, 1] = coord[:, 1] + new_coord[:, 3] / 2 # y center
    return new_coord

In [13]:
from torch.nn import functional as F
class AssignmentSimilarityNet(nn.Module):
    def __init__(self, reid_network, node_dim, edge_dim, reid_dim, edges_in_dim, num_steps, dropout=0.):
        super().__init__()
        self.reid_network = reid_network
        self.graph_net = BipartiteNeuralMessagePassingLayer(node_dim=node_dim, edge_dim=edge_dim, dropout=dropout)
        self.num_steps = num_steps
        self.cnn_linear = nn.Linear(reid_dim, node_dim)
        self.edge_in_mlp = nn.Sequential(*[nn.Linear(edges_in_dim, edge_dim), nn.ReLU(), nn.Dropout(dropout), nn.Linear(edge_dim, edge_dim), nn.ReLU(),nn.Dropout(dropout)])
        self.classifier = nn.Sequential(*[nn.Linear(edge_dim, edge_dim), nn.ReLU(), nn.Linear(edge_dim, 1)])
        
    
    def compute_edge_feats(self, track_coords, current_coords, track_t, curr_t):    
        """
        Computes initial edge feature tensor

        Args:
            track_coords: track's frame box coordinates, given by top-left and bottom-right coordinates
                          torch.Tensor with shape (num_tracks, 4)
            current_coords: current frame box coordinates, given by top-left and bottom-right coordinates
                            has shape (num_boxes, 4)
                          
            track_t: track's timestamps, torch.Tensor with with shape (num_tracks, )
            curr_t: current frame's timestamps, torch.Tensor withwith shape (num_boxes,)        
            
        
        Returns:
            tensor with shape (num_trakcs, num_boxes, 5) containing pairwise
            position and time difference features 
        """

        ########################
        #### TODO starts
        ########################
        
        # NOTE 1: we recommend you to use box centers to compute distances
        # in the x and y coordinates.

        # NOTE 2: Check out the  code inside train_one_epoch function and 
        # LongTrackTrainingDataset class a few cells below to debug this
        edge_feats = torch.zeros((track_coords.shape[0], current_coords.shape[0], 5)).cuda()
        track_coords1 = change_coords(track_coords).cuda()
        current_coords1 = change_coords(current_coords).cuda()

        edge_feats[:, :, 0] = (2 * (current_coords1[:, 0].unsqueeze(1) - track_coords1[:, 0]) / 
                               (current_coords1[:, 3].unsqueeze(1) + track_coords1[:, 3])).T
        
        edge_feats[:, :, 1] = (2 * (current_coords1[:, 1].unsqueeze(1) - track_coords1[:, 1]) / 
                               (current_coords1[:, 3].unsqueeze(1) + track_coords1[:, 3])).T

        edge_feats[:, :, 2] = torch.log(torch.div(track_coords1[:, 3].unsqueeze(1), current_coords1[:, 3]))

        edge_feats[:, :, 3] = torch.log(torch.div(track_coords1[:, 2].unsqueeze(1), current_coords1[:, 2]))

        edge_feats[:, :, 4] = (curr_t.unsqueeze(1) - track_t).T
        ########################
        #### TODO ends
        ########################

        return edge_feats # has shape (num_tracks, num_boxes, 5)


    def forward(self, track_app, current_app, track_coords, current_coords, track_t, curr_t):
        """
        Args:
            track_app: track's reid embeddings, torch.Tensor with shape (num_tracks, 512)
            current_app: current frame detections' reid embeddings, torch.Tensor with shape (num_boxes, 512)
            track_coords: track's frame box coordinates, given by top-left and bottom-right coordinates
                          torch.Tensor with shape (num_tracks, 4)
            current_coords: current frame box coordinates, given by top-left and bottom-right coordinates
                            has shape (num_boxes, 4)
                          
            track_t: track's timestamps, torch.Tensor with with shape (num_tracks, )
            curr_t: current frame's timestamps, torch.Tensor withwith shape (num_boxes,)
            
        Returns:
            classified edges: torch.Tensor with shape (num_steps, num_tracks, num_boxes),
                             containing at entry (step, i, j) the unnormalized probability that track i and 
                             detection j are a match, according to the classifier at the given neural message passing step
        """

        # Get initial edge embeddings
        dist_reid = cosine_distance(track_app, current_app)
        pos_edge_feats = self.compute_edge_feats(track_coords, current_coords, track_t, curr_t)
        edge_feats = torch.cat((pos_edge_feats, dist_reid.unsqueeze(-1)), dim=-1)
        edge_embeds = self.edge_in_mlp(edge_feats)
        initial_edge_embeds = edge_embeds.clone()

        # Get initial node embeddings, reduce dimensionality from 512 to node_dim
        track_embeds = F.relu(self.cnn_linear(track_app))
        curr_embeds = F.relu(self.cnn_linear(current_app))

        classified_edges = []
        for _ in range(self.num_steps):
            edge_embeds = torch.cat((edge_embeds, initial_edge_embeds), dim=-1)            
            edge_embeds, track_embeds, curr_embeds = self.graph_net(edge_embeds=edge_embeds, 
                                                                    nodes_a_embeds=track_embeds, 
                                                                    nodes_b_embeds=curr_embeds)

            classified_edges.append(self.classifier(edge_embeds))

        return torch.stack(classified_edges).squeeze(-1)

## Putting everything together

Finally, we incorporate our ``AssignmentSimilarityNet`` into our tracker. We can keep everything as in ``LongTermReIDHungarianTracker`` except for the distance computation, which is now directly obtained via a forward pass through AssignmentSimilarityNet.

In [14]:
_UNMATCHED_COST=255
class MPNTracker(LongTermReIDHungarianTracker):
    def __init__(self, assign_net, *args, **kwargs):
        self.assign_net = assign_net
        super().__init__(*args, **kwargs)
        
    def data_association(self, boxes, scores, pred_features):  
        if self.tracks:  
            track_boxes = torch.stack([t.box for t in self.tracks], axis=0).cuda()
            track_features = torch.stack([t.get_feature() for t in self.tracks], axis=0).cuda()
            
            # Hacky way to recover the timestamps of boxes and tracks
            curr_t = self.im_index * torch.ones((pred_features.shape[0],)).cuda()
            track_t = torch.as_tensor([self.im_index - t.inactive - 1 for t in self.tracks]).cuda()

            ########################
            #### TODO starts
            ########################
            
            # Do a forward pass through self.assign_net to obtain our costs.
            # Note: self.assign_net will return unnormalized probabilities. 
            # Make sure to apply the sigmoid function to them!
            pred_features = pred_features.cuda()
            pred_sim = self.assign_net(track_features, pred_features, track_boxes, boxes, track_t, curr_t)
            pred_sim = torch.sigmoid(pred_sim).cpu()
            pred_features = pred_features.cpu()
            ########################
            #### TODO ends
            ########################

            pred_sim = pred_sim[-1]  # Use predictions at last message passing step
            distance = (1- pred_sim) 
            
            # Do not allow mataches when sim < 0.5, to avoid low-confident associations
            distance = np.where(pred_sim < 0.5, _UNMATCHED_COST, distance) 

            # Perform Hungarian matching.
            row_idx, col_idx = linear_assignment(distance)            
            self.update_tracks(row_idx, col_idx, distance, boxes, scores, pred_features)

            
        else:
            # No tracks exist.
            self.add(boxes, scores, pred_features)

## Training and evaluating our model

We provide all boilerplate code for training our neural message passing based
tracker, as well as evaluating. 

Under the hood, we are sampling frames randomly from our training sequences, and then sampling boxes from past frames as past_tracks to generate our 
training
data. Check out `LongTrackTrainingDataset` for details.

We train the model with a weighted cross-entropy loss
to account for the class imbalance. Check out `train_one_epoch` if you're 
interested.

Since the data is unbalanced, we add a weight $w$ for the positive samples in the binary cross entropy loss.
$$ \sum_{{i,j}} w \cdot y_{i,j} \log\left(\hat{y_{i,j}} \right) + \left(1-y_{i,j}\right)\log\left(1- \hat{y_{i,j}}\right), $$
where $$y_{i,j} \in\{ 0,1\}$$ is one, if the edge is active in the training data and zero, otherwise.
The variable $$\hat{y_{i,j}} \in[0,1]$$ denotes the learned output probability of the edge being active, which is later used for classification.


No need to write any code from your side here!


In [15]:
from gnn.dataset import LongTrackTrainingDataset
from torch.utils.data import DataLoader
from gnn.trainer import train_one_epoch

MAX_PATIENCE = 20
MAX_EPOCHS = 15
EVAL_FREQ = 1


# Define our model, and init 
assign_net = AssignmentSimilarityNet(reid_network=None, # Not needed since we work with precomputed features
                                     node_dim=32, 
                                     edge_dim=64, 
                                     reid_dim=512, 
                                     edges_in_dim=6, 
                                     num_steps=10).cuda()

# We only keep two sequences for validation
dataset = LongTrackTrainingDataset(dataset='MOT16-train_wo_val2', 
                                   db=train_db, 
                                   root_dir= osp.join(root_dir, 'data/MOT16'),
                                   max_past_frames = MAX_PATIENCE,
                                   vis_threshold=0.25)

data_loader = DataLoader(dataset, batch_size=8, collate_fn = lambda x: x, 
                         shuffle=True, num_workers=2, drop_last=True)
device = torch.device('cuda')
optimizer = torch.optim.Adam(assign_net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5)

We only leave 2 sequences for validation in order to maximize 
the amount of training data. For your convenience, here are the
 LongTermReIDTracker results on them. Your validation IDF1 scores should show an improvement over them.
```

          IDF1   IDP   IDR  Rcll  Prcn  GT MT PT ML  FP    FN IDs   FM  MOTA  MOTP
MOT16-02 45.9% 65.1% 35.4% 52.2% 96.1%  62 12 37 13 390  8873 130  210 49.4% 0.090
MOT16-11 68.3% 75.3% 62.5% 80.2% 96.6%  75 44 24  7 266  1871 136   90 75.9% 0.083
OVERALL  54.3% 69.6% 44.5% 61.7% 96.3% 137 56 61 20 656 10744 266  300 58.4% 0.087
```



Let's start training!

Note that we have observed quite a lot of noise in validation scores among epochs and runs. That is due to the small size of our training and validation sets. Therefore, we perform early stopping to obtain the best-performing model on validation. In addition, changing the experiment seed and/ or relaunching the training might help if you suspect noise influencing your scores. 

In [16]:
best_idf1 = 0.
for epoch in range(1, MAX_EPOCHS + 1):
    print(f"-------- EPOCH {epoch:2d} --------")
    train_one_epoch(model = assign_net, data_loader=data_loader, optimizer=optimizer, print_freq=100)
    scheduler.step()

    if epoch % EVAL_FREQ == 0:
        tracker =  MPNTracker(assign_net=assign_net.eval(), obj_detect=None, patience=MAX_PATIENCE)
        val_sequences = MOT16Sequences('MOT16-val2', osp.join(root_dir, 'data/MOT16'), vis_threshold=0.)
        res = run_tracker(val_sequences, db=train_db, tracker=tracker, output_dir=None)
        idf1 = res.loc['OVERALL']['idf1']
        if idf1 > best_idf1:
            best_idf1 = idf1
            torch.save(assign_net.state_dict(), osp.join(root_dir, 'output', 'best_ckpt.pth'))
        

-------- EPOCH  1 --------


100it [00:29,  3.91it/s]

Iter 100. Loss: 0.605. Accuracy: 0.951. Recall: 0.762. Precision: 0.584


200it [00:54,  3.85it/s]

Iter 200. Loss: 0.044. Accuracy: 0.998. Recall: 0.994. Precision: 0.976


...

Iter 400. Loss: 0.016. Accuracy: 0.999. Recall: 0.998. Precision: 0.987


476it [02:04,  3.83it/s]


Tracking: MOT16-02
Tracks found: 99
Runtime for MOT16-02: 8.8 s.
Tracking: MOT16-11
Tracks found: 84
Runtime for MOT16-11: 8.0 s.
Runtime for all sequences: 16.8 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT MT PT ML  FP    FN IDs   FM  MOTA  MOTP
MOT16-02 45.4% 64.4% 35.0% 52.2% 96.1%  62 11 38 13 390  8873 110  217 49.6% 0.095
MOT16-11 70.4% 77.7% 64.4% 80.2% 96.6%  75 44 24  7 266  1871  33   90 77.0% 0.083
OVERALL  54.8% 70.2% 44.9% 61.7% 96.3% 137 55 62 20 656 10744 143  307 58.8% 0.090
-------- EPOCH  3 --------


77it [00:20,  3.67it/s]


KeyboardInterrupt: ignored

# Exercise submission

After executing the following cell, the `Colab Notebooks/cv3dst_exercise/output` directory in your Google Drive should contain multiple `MOT16-XY.txt` files.

For the final submission, you must process the test sequences and upload the zipped prediction files to our server.

Note that this time, you only have to evaluate three sequences!

In [ ]:
best_ckpt = torch.load(osp.join(root_dir, 'output', 'best_ckpt.pth'))
assign_net.load_state_dict(best_ckpt)

tracker =  MPNTracker(assign_net=assign_net.eval(), obj_detect=None, patience=MAX_PATIENCE)
test_db = torch.load(osp.join(gnn_root_dir, 'data/preprocessed_data_test_2.pth'))
val_sequences = MOT16Sequences('MOT16-test', osp.join(root_dir, 'data/MOT16'), vis_threshold=0.)
run_tracker(val_sequences, db=test_db, tracker=tracker, output_dir=osp.join(root_dir, 'output'))

Tracking: MOT16-01
No GT evaluation data available.
Tracks found: 78
Runtime for MOT16-01: 3.6 s.
Writing predictions to: gdrive/My Drive/Colab Notebooks/cv3dst_exercise/output/MOT16-01.txt
Tracking: MOT16-08
No GT evaluation data available.
Tracks found: 141
Runtime for MOT16-08: 5.0 s.
Writing predictions to: gdrive/My Drive/Colab Notebooks/cv3dst_exercise/output/MOT16-08.txt
Tracking: MOT16-12
No GT evaluation data available.
Tracks found: 144
Runtime for MOT16-12: 6.5 s.
Writing predictions to: gdrive/My Drive/Colab Notebooks/cv3dst_exercise/output/MOT16-12.txt
Runtime for all sequences: 15.1 s.
